In [1]:
import pandas as pd
import numpy as np

df_all = pd.read_csv("cortical_metrics_1.4.csv")
area = 'VISp'

# Exc and PV have sufficient number of cells, so we'll filter out non-V1 Exc and PV.
# SST and VIP are small populations, so let's keep also non-V1 neurons
exclude = (df_all["cell_type"].isnull() | df_all["cell_type"].str.contains("EXC") | df_all["cell_type"].str.contains("PV")) \
        & (df_all["ecephys_structure_acronym"] != area)

df = df_all[~exclude]
print(f"Original: {df_all.shape[0]} cells,   filtered: {df.shape[0]} cells")

/tmp/ipykernel_2775405/3334403599.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_2775405/3334403599.py:4: DtypeWarning: Columns (60,62,65,67,69,72,119,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv("cortical_metrics_1.4.csv")


Original: 40294 cells,   filtered: 8799 cells


In [2]:
# Count the number of observation from every session type
bo_counts = df[df["session_type"] == "brain_observatory_1.1"]["cell_type"].value_counts()
fc_counts = df[df["session_type"] == "functional_connectivity"]["cell_type"].value_counts()

# Some cells have very large values of RF. They are likely not-good fits, so ignore.
df.loc[(df["width_rf"] > 100), "width_rf"] = np.nan
df.loc[(df["height_rf"] > 100), "height_rf"] = np.nan

# define contrast response types
types = ["high", "band", "low"]
type_fracs = {}
type_fracs_err = {}
contrast_responsive = df["sig0.01"] == True
contrast_responsive_counts = df[contrast_responsive]["cell_type"].value_counts()
for t in types:
    type_counts = df[(df["best_model"] == t) & contrast_responsive]["cell_type"].value_counts()
    n = contrast_responsive_counts
    p = type_counts / n
    type_fracs[t] = p
    type_fracs_err[t] = np.sqrt(p * (1 - p) / n)

# Save the processed table
df.to_csv("v1_OSI_DSI_DF.csv", sep=" ", index=False)

In [5]:
# count the number of units of each cell_type
cell_type_counts = df["cell_type"].value_counts()
cell_type_counts

cell_type
EXC_L5     975
EXC_L4     629
EXC_L23    503
EXC_L6     344
PV_L5      200
PV_L4      179
ALL_L1     141
PV_L23     133
PV_L6       90
SST_L5      27
SST_L23     18
SST_L6      16
SST_L4       9
VIP_L23      5
VIP_L5       5
VIP_L6       3
VIP_L4       3
Name: count, dtype: int64

In [54]:
df

,ecephys_unit_id,L_ratio,age_in_days,amplitude_cutoff,anterior_posterior_ccf_coordinate,area_rf,azimuth_rf,c50_dg,cumulative_drift,d_prime,...,aic_diff1,aic_diff2,resp_sig,sig0.01,firing_rate_sp,preferred_angle,max_mean_rate(Hz),Ave_Rate(Hz),OSI,DSI
154,915960235,0.00,100.00,0.50,NaN,700.00,77.14,NaN,1043.19,5.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,915960239,0.04,100.00,0.50,NaN,500.00,18.00,NaN,213.33,2.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,915960244,0.02,100.00,0.47,NaN,1000.00,36.00,NaN,195.20,2.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,915960249,0.03,100.00,0.31,NaN,1100.00,58.18,NaN,369.14,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,915960253,0.01,100.00,0.10,NaN,1900.00,56.84,NaN,234.91,3.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40289,951190691,0.03,126.00,0.01,7318.00,NaN,NaN,0.36,27.31,4.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40290,951190693,NaN,126.00,0.11,7318.00,NaN,NaN,0.36,0.00,3.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40291,951190812,0.00,126.00,0.24,7318.00,NaN,NaN,0.36,0.00,5.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40292,951190814,0.03,126.00,0.45,7318.00,NaN,NaN,0.36,8.26,4.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# do median and ignore non numerical columns
df.groupby(["cell_type", "session_type"]).median(numeric_only=True)

ecephys_unit_id  L_ratio  age_in_days  \
cell_type session_type                                                     
ALL_L1    brain_observatory_1.1       951846255.00     0.03       103.00   
          functional_connectivity     951172552.00     0.01       126.00   
EXC_L23   brain_observatory_1.1       951096075.00     0.01       110.00   
          functional_connectivity     951134148.00     0.00       121.00   
EXC_L4    brain_observatory_1.1       951095642.00     0.01       116.00   
          functional_connectivity     951128248.00     0.00       121.00   
EXC_L5    brain_observatory_1.1       951110061.50     0.01       108.50   
          functional_connectivity     951133606.00     0.00       121.00   
EXC_L6    brain_observatory_1.1       951108620.00     0.01       110.00   
          functional_connectivity     951128109.00     0.00       121.00   
PV_L23    brain_observatory_1.1       951811243.00     0.01       105.00   
          functional_connectivity     951131261.50     0.01       123.50   
PV_L4     brain_observatory_1.1       951811074.50     0.01       103.00   
          functional_connectivity     951134083.00     0.00       121.00   
PV_L5     brain_observatory_1.1       951108850.00     0.00       116.00   
          functional_connectivity     951128219.00     0.00       121.00   
PV_L6     brain_observatory_1.1       951439805.00     0.00       119.50   
          functional_connectivity     951128117.00     0.00       121.00   
SST_L23   brain_observatory_1.1       950956845.00     0.00       118.00   
          functional_connectivity     951093033.00     0.00       120.00   
SST_L4    brain_observatory_1.1       951882662.50     0.00       106.00   
          functional_connectivity     951088165.00     0.00       129.00   
SST_L5    brain_observatory_1.1       951879166.50     0.02       106.00   
          functional_connectivity     951087454.00     0.00       120.00   
SST_L6    brain_observatory_1.1       950930145.00     0.00       118.00   
          functional_connectivity     951096315.00     0.01       129.00   
VIP_L23   brain_observatory_1.1       951064902.00     0.04       116.00   
VIP_L4    brain_observatory_1.1       951116761.00     0.00       110.00   
          functional_connectivity     951140966.00     0.01       135.00   
VIP_L5    brain_observatory_1.1       951085671.00     0.01       113.00   
          functional_connectivity     951165437.00     0.00       121.00   
VIP_L6    brain_observatory_1.1       951112448.00     0.03       110.00   
          functional_connectivity     951140952.00     0.00       135.00   

                                   amplitude_cutoff  \
cell_type session_type                                
ALL_L1    brain_observatory_1.1                0.13   
          functional_connectivity              0.03   
EXC_L23   brain_observatory_1.1                0.01   
          functional_connectivity              0.00   
EXC_L4    brain_observatory_1.1                0.01   
          functional_connectivity              0.00   
EXC_L5    brain_observatory_1.1                0.01   
          functional_connectivity              0.00   
EXC_L6    brain_observatory_1.1                0.01   
          functional_connectivity              0.00   
PV_L23    brain_observatory_1.1                0.00   
          functional_connectivity              0.02   
PV_L4     brain_observatory_1.1                0.02   
          functional_connectivity              0.01   
PV_L5     brain_observatory_1.1                0.00   
          functional_connectivity              0.00   
PV_L6     brain_observatory_1.1                0.01   
          functional_connectivity              0.00   
SST_L23   brain_observatory_1.1                0.01   
          functional_connectivity              0.00   
SST_L4    brain_observatory_1.1                0.02   
          functional_connectivity              0.03   
SST_L5    brain_observatory_1.1                0.01   

In [44]:
df[['cell_type','anterior_posterior_ccf_coordinate']].groupby("cell_type").median()

,anterior_posterior_ccf_coordinate
cell_type,
ALL_L1,7608.00
EXC_L23,8882.00
EXC_L4,8808.00
EXC_L5,8761.00
EXC_L6,8700.00
PV_L23,8879.00
PV_L4,8816.50
PV_L5,8752.50
PV_L6,8674.00


In [47]:
# set group statistics, and insert the cell counts for convenience
resource_tables = {}
resource_tables["median"] = df.groupby("cell_type").median(numeric_only=True)
resource_tables["mean"] = df.groupby("cell_type").mean(numeric_only=True)
resource_tables["sem"] = df.groupby("cell_type").sem(numeric_only=True)

for t in resource_tables.values():
    # insert counts and fractional elements that is common for all 3 tables
    t.insert(0, "n_cells_BO", bo_counts)
    t.insert(1, "n_cells_FC", fc_counts)
    t["n_cells_FC"][t["n_cells_FC"].isna()] = 0
    t["n_cells_FC"] = t["n_cells_FC"].astype(dtype=int)

    # t.insert(-1, "frac_highpass", type_fracs["high"])
    t["frac_highpass"] = type_fracs["high"]
    t["frac_bandpass"] = type_fracs["band"]
    t["frac_lowpass"] = type_fracs["low"]

# sem is a special case. re-insert the estimated error of the fraction
resource_tables["sem"]["frac_highpass"] = type_fracs_err["high"]
resource_tables["sem"]["frac_bandpass"] = type_fracs_err["band"]
resource_tables["sem"]["frac_lowpass"] = type_fracs_err["low"]

/tmp/ipykernel_989282/3857521699.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  t["n_cells_FC"][t["n_cells_FC"].isna()] = 0
/tmp/ipykernel_989282/3857521699.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [48]:
selected_metrics = [
    "n_cells_BO",
    "n_cells_FC",
    "g_osi_dg",
    "g_dsi_dg",
    "pref_sf_sg",
    "pref_tf_dg",
    "firing_rate_sp",
    "firing_rate_rf",
    "firing_rate_dg",
    "width_rf",
    "height_rf",
    "fano_dg",
    "frac_highpass",
    "frac_bandpass",
    "frac_lowpass",
]

# For the full list of metrics, refer to list(df.columns)
#
# See https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html
# for detailed descriptions of the metrics.


In [49]:
pd.set_option("display.float_format", "{:0.2f}".format)
print('Median functional responses')
resource_tables["median"][selected_metrics]

Median functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.18,0.11,0.04,2.00,2.21,0.00,0.40,25.90,28.65,3.37,0.83,0.17,NaN
EXC_L23,49,67,0.27,0.14,0.04,2.00,0.81,0.62,0.84,16.38,18.69,3.16,0.64,0.25,0.11
EXC_L4,89,190,0.17,0.11,0.04,2.00,1.85,1.37,2.09,20.28,19.59,3.76,0.53,0.31,0.16
EXC_L5,328,449,0.20,0.09,0.08,2.00,2.95,2.57,3.21,32.81,37.10,3.79,0.71,0.21,0.08
EXC_L6,106,155,0.18,0.08,0.08,2.00,2.18,1.38,2.76,41.92,48.06,4.15,0.77,0.16,0.07
PV_L23,13,30,0.12,0.09,0.04,2.00,5.41,3.34,6.55,15.56,21.73,7.01,0.80,0.16,0.04
PV_L4,12,40,0.14,0.11,0.02,2.00,6.49,4.35,8.08,17.81,20.61,5.99,0.70,0.20,0.10
PV_L5,45,63,0.12,0.06,0.08,2.00,8.17,6.20,12.06,31.09,37.76,6.41,0.79,0.21,NaN
PV_L6,16,23,0.10,0.04,0.04,2.00,11.70,11.37,18.46,49.65,61.43,5.19,0.93,0.07,NaN


In [50]:
print('Mean functional responses')
resource_tables["mean"][selected_metrics]


Mean functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.33,0.26,0.06,2.34,4.61,0.77,1.34,2.35,16.26,4.28,0.83,0.17,NaN
EXC_L23,49,67,0.34,0.16,0.08,2.77,1.57,1.55,1.98,18.68,21.87,4.74,0.64,0.25,0.11
EXC_L4,89,190,0.21,0.13,0.09,3.02,3.27,2.58,3.26,22.92,21.20,4.57,0.53,0.31,0.16
EXC_L5,328,449,0.24,0.11,0.10,3.30,4.71,4.37,4.96,35.82,38.18,5.12,0.71,0.21,0.08
EXC_L6,106,155,0.27,0.10,0.12,3.32,3.28,2.52,3.71,42.02,44.25,5.63,0.77,0.16,0.07
PV_L23,13,30,0.12,0.08,0.06,2.14,6.79,5.37,8.04,20.63,24.13,8.47,0.80,0.16,0.04
PV_L4,12,40,0.18,0.11,0.03,2.17,8.94,5.56,8.97,20.94,21.94,7.48,0.70,0.20,0.10
PV_L5,45,63,0.14,0.09,0.08,3.20,10.99,9.67,14.99,33.41,38.97,8.27,0.79,0.21,NaN
PV_L6,16,23,0.13,0.04,0.05,2.67,13.50,14.16,21.60,53.01,54.40,7.65,0.93,0.07,NaN


In [51]:
print("SEM of functional responses")
resource_tables["sem"][selected_metrics]

SEM of functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.04,0.04,0.01,0.20,1.11,0.17,0.19,26.06,14.63,0.39,0.15,0.15,NaN
EXC_L23,49,67,0.04,0.02,0.01,0.30,0.24,0.20,0.26,4.18,4.54,0.46,0.08,0.07,0.05
EXC_L4,89,190,0.02,0.01,0.01,0.20,0.23,0.21,0.20,5.99,6.61,0.18,0.05,0.04,0.04
EXC_L5,328,449,0.01,0.01,0.01,0.14,0.17,0.19,0.17,1.12,1.28,0.18,0.03,0.03,0.02
EXC_L6,106,155,0.02,0.01,0.01,0.24,0.21,0.19,0.22,2.52,2.52,0.30,0.05,0.04,0.03
PV_L23,13,30,0.02,0.01,0.01,0.21,1.06,0.82,0.90,2.28,1.66,0.80,0.08,0.07,0.04
PV_L4,12,40,0.03,0.02,0.01,0.18,1.01,0.68,0.82,2.30,3.96,0.85,0.08,0.07,0.05
PV_L5,45,63,0.01,0.01,0.01,0.36,0.95,1.17,1.24,2.47,3.00,0.66,0.06,0.06,NaN
PV_L6,16,23,0.03,0.01,0.01,0.40,1.65,2.16,2.26,7.16,6.17,1.27,0.06,0.06,NaN
